### New GPU Memory Configuration

In [1]:
# ==============================================================================
# 0. GPU Configuration (NEW)
# ==============================================================================
# Configure TensorFlow to grow GPU memory usage as needed. This can help prevent
# out-of-memory errors on some systems.
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU memory growth configured for {len(gpus)} device(s).")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2025-09-22 05:24:29.144045: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPU memory growth configured for 1 device(s).


### Dependencies & Global Configuration

In [2]:
# ==============================================================================
# 1. Importing Dependencies & Configuration
# ==============================================================================
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import pywt
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import json
from typing import List, Tuple, Dict, Optional

# Keras and TensorFlow Layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, Reshape, Bidirectional, LSTM, Dropout, Dense,
    TimeDistributed, GlobalAveragePooling1D, concatenate, Layer
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

# Scikit-learn for data splitting and class weights
try:
    from sklearn.model_selection import train_test_split
    from sklearn.utils.class_weight import compute_class_weight
    _HAVE_SKLEARN = True
except ImportError:
    _HAVE_SKLEARN = False

# --- Global Constants ---
CLASS_NAMES = [
    'happy', 'sad', 'surprised', 'satisfied',
    'protected', 'frightened', 'angry', 'unconcerned'
]

FOLDER_TO_CLASS = {
    'Happy': 'happy',
    'Sad': 'sad',
    'Surprise': 'surprised',
    'Satisfied': 'satisfied',
    'Protected': 'protected',
    'Frightened': 'frightened',
    'Angry': 'angry',
    'Unconcerned': 'unconcerned'
}

# The 14 main EEG channels to be used from the CSV files
EEG_CHANNELS = [
    'EEG.AF3', 'EEG.F7', 'EEG.F3', 'EEG.FC5', 'EEG.T7', 'EEG.P7', 'EEG.O1',
    'EEG.O2', 'EEG.P8', 'EEG.T8', 'EEG.FC6', 'EEG.F4', 'EEG.F8', 'EEG.AF4'
]

### Data Preprocessing & Loading Functions

In [3]:
# ==============================================================================
# 2. Data Loading and Preprocessing Functions (Corrected for Sample Weights)
# ==============================================================================

def wavelet_denoise(data, wavelet='db4', level=4):
    """Applies wavelet denoising to a 1D signal."""
    coeffs = pywt.wavedec(data, wavelet, level=level)
    sigma = np.median(np.abs(coeffs[-1] - np.median(coeffs[-1]))) / 0.6745
    threshold = sigma * np.sqrt(2 * np.log(len(data)))
    new_coeffs = coeffs.copy()
    for i in range(1, len(coeffs)):
        new_coeffs[i] = pywt.threshold(coeffs[i], value=threshold, mode='soft')
    reconstructed_signal = pywt.waverec(new_coeffs, wavelet)
    return reconstructed_signal[:len(data)]

def _read_and_denoise_csv(filepath: str) -> np.ndarray:
    """
    Reads a CSV file, applies wavelet denoising to EEG channels,
    and returns a NumPy array of all numeric columns.
    """
    df = pd.read_csv(filepath, skiprows=1, low_memory=False)
    for channel in EEG_CHANNELS:
        if channel in df.columns:
            signal = df[channel].dropna().values
            if np.var(signal) > 0:
                denoised_signal = wavelet_denoise(signal)
                df.loc[:len(denoised_signal)-1, channel] = denoised_signal
    numeric_df = df.select_dtypes(include=[np.number]).copy().fillna(0)
    arr = numeric_df.values.astype(np.float32)
    return arr[:, None] if arr.ndim == 1 else arr

def _ensure_shape_and_resample(raw: np.ndarray, time_steps: int, channels: int, features: int) -> np.ndarray:
    """Ensure the data has the correct shape by padding or truncating."""
    flat = raw.flatten()
    needed = time_steps * channels * features
    if flat.size >= needed:
        flat = flat[:needed]
    else:
        pad = np.zeros(needed - flat.size, dtype=flat.dtype)
        flat = np.concatenate([flat, pad], axis=0)
    return flat.reshape((time_steps, channels, features))

def _normalize_per_sample(sample: np.ndarray) -> np.ndarray:
    """Normalize each sample by subtracting mean and dividing by standard deviation."""
    mean = sample.mean(axis=0, keepdims=True)
    std = sample.std(axis=0, keepdims=True)
    std[std < 1e-8] = 1.0
    return (sample - mean) / std

def load_eeg_dataset(
    data_dir: str, time_steps: int, channels: int, features: int,
    stressed_classes: Optional[List[str]] = None, test_size: float = 0.15,
    val_size: float = 0.15, random_state: int = 42, batch_size: int = 4
) -> Tuple[Dict[str, tf.data.Dataset], Dict]:
    """Load EEG dataset with sample weights included directly."""
    if not _HAVE_SKLEARN:
        raise ImportError("Scikit-learn is required. Please run 'pip install scikit-learn'.")

    if stressed_classes is None: stressed_classes = ['frightened', 'angry']
    files, labels = [], []
    for folder, cls in FOLDER_TO_CLASS.items():
        cls_folder = os.path.join(data_dir, folder)
        if os.path.isdir(cls_folder):
            found = glob.glob(os.path.join(cls_folder, "*.csv"))
            files.extend(found)
            labels.extend([cls] * len(found))

    if not files: raise ValueError(f"No CSV files found in subdirectories of {data_dir}.")

    X_list, y_multi_idx, y_binary = [], [], []
    for fpath, cls in zip(files, labels):
        raw = _read_and_denoise_csv(fpath)
        sample = _ensure_shape_and_resample(raw, time_steps, channels, features)
        sample = _normalize_per_sample(sample)
        X_list.append(sample.astype(np.float32))
        y_multi_idx.append(CLASS_NAMES.index(cls))
        y_binary.append(1 if cls in stressed_classes else 0)

    X = np.stack(X_list, axis=0)
    y_multi_idx = np.array(y_multi_idx, dtype=np.int32)
    y_binary = np.array(y_binary, dtype=np.float32)
    y_multi_onehot = tf.keras.utils.to_categorical(y_multi_idx, num_classes=len(CLASS_NAMES))
    
    weights = compute_class_weight('balanced', classes=np.arange(len(CLASS_NAMES)), y=y_multi_idx)
    class_weights = {i: float(w) for i, w in enumerate(weights)}
    
    # *** FIX IS HERE: Create sample weights for each data point ***
    emotion_sample_weights = np.array([class_weights[label] for label in y_multi_idx], dtype=np.float32)
    stress_sample_weights = np.ones_like(y_binary, dtype=np.float32) # No special weights for stress
    
    indices = np.arange(len(X))
    train_indices, temp_indices = train_test_split(indices, test_size=(test_size + val_size), random_state=random_state, stratify=y_multi_idx)
    val_indices, test_indices = train_test_split(temp_indices, test_size=(test_size / (test_size + val_size)), random_state=random_state, stratify=y_multi_idx[temp_indices])

    def make_ds(inds):
        x = X[inds]
        y = {'stressed_not_stressed_output': y_binary[inds], 'emotion_class_output': y_multi_onehot[inds]}
        # Create a matching dictionary for sample weights
        sw = {'stressed_not_stressed_output': stress_sample_weights[inds], 'emotion_class_output': emotion_sample_weights[inds]}
        # Include sample weights as the third element
        return tf.data.Dataset.from_tensor_slices((x, y, sw)).shuffle(len(inds), seed=random_state).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    datasets = {'train': make_ds(train_indices), 'val': make_ds(val_indices), 'test': make_ds(test_indices)}
    meta = {'counts': {cls: labels.count(cls) for cls in CLASS_NAMES}, 'total_samples': len(X), 'class_weights': class_weights, 'index_to_class': {i: c for i, c in enumerate(CLASS_NAMES)}}
    return datasets, meta

### Model Architecture

In [4]:
# ==============================================================================
# 3. Model Definition (Lighter Version)
# ==============================================================================

class Attention(Layer):
    # ... (This class remains unchanged) ...
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(Attention, self).build(input_shape)
    def call(self, x):
        et = tf.keras.backend.squeeze(tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b), axis=-1)
        at = tf.keras.backend.softmax(et)
        at = tf.keras.backend.expand_dims(at, axis=-1)
        output = x * at
        return tf.keras.backend.sum(output, axis=1)

def create_eeg_model(input_shape):
    """Create a LIGHTER multi-output model to consume less memory."""
    input_layer = Input(shape=input_shape)
    x = tf.keras.layers.Lambda(lambda z: tf.expand_dims(z, axis=-1))(input_layer)
    
    # *** FIX: Reduced filters in Conv2D layers ***
    x = TimeDistributed(Conv2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(Conv2D(filters=4, kernel_size=(3, 3), activation='relu', padding='same'))(x)
    
    x = Reshape((input_shape[0], -1))(x)

    # *** FIX: Reduced units in LSTM layers ***
    x = Bidirectional(LSTM(16, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = Bidirectional(LSTM(8, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    
    attention_output = Attention()(x)
    main_path = GlobalAveragePooling1D()(x)
    main_path = concatenate([main_path, attention_output])

    # Binary head (Stress)
    binary_head = Dense(16, activation='relu')(main_path) # Reduced units
    binary_head = Dropout(0.4)(binary_head)
    binary_head_output = Dense(1, activation='sigmoid', name='stressed_not_stressed_output')(binary_head)

    # Multi-class head (Emotion)
    multiclass_head = Dense(16, activation='relu')(main_path) # Reduced units
    multiclass_head = Dropout(0.4)(multiclass_head)
    multiclass_head_output = Dense(len(CLASS_NAMES), activation='softmax', name='emotion_class_output')(multiclass_head)

    model = Model(
        inputs=input_layer,
        outputs={
            "stressed_not_stressed_output": binary_head_output,
            "emotion_class_output": multiclass_head_output
        }
    )
    return model

### Main Execution and Training

In [5]:
# ==============================================================================
# 4. Main Execution Block (Corrected Callbacks)
# ==============================================================================
if __name__ == '__main__':
    # Add the GPU configuration from the previous step here
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"GPU memory growth configured for {len(gpus)} device(s).")
        except RuntimeError as e:
            print(e)

    dataset_path = "/media/kd/New Volume/Github/EEG-Emotion-Detection/dataset"
    
    INPUT_TIME_STEPS = 128
    INPUT_CHANNELS = 55
    INPUT_FEATURES = 24
    INPUT_3D_SHAPE = (INPUT_TIME_STEPS, INPUT_CHANNELS, INPUT_FEATURES)

    # Use the smaller batch size
    datasets, meta = load_eeg_dataset(
        dataset_path,
        time_steps=INPUT_TIME_STEPS,
        channels=INPUT_CHANNELS,
        features=INPUT_FEATURES,
        batch_size=4 
    )

    # Use the lighter model
    model = create_eeg_model(INPUT_3D_SHAPE)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={'stressed_not_stressed_output': 'binary_crossentropy', 'emotion_class_output': 'categorical_crossentropy'},
        loss_weights={'stressed_not_stressed_output': 0.5, 'emotion_class_output': 1.0},
        metrics={'stressed_not_stressed_output': 'accuracy', 'emotion_class_output': 'accuracy'}
    )
    model.summary()

    os.makedirs('models', exist_ok=True)
    timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    # *** FIX IS HERE: Added mode='max' to EarlyStopping and ReduceLROnPlateau ***
    callbacks = [
        ModelCheckpoint(
            filepath=f'models/best_model_{timestamp}.h5', 
            monitor='val_emotion_class_output_accuracy', 
            save_best_only=True, 
            mode='max', # This was already correct
            verbose=1
        ),
        EarlyStopping(
            monitor='val_emotion_class_output_accuracy', 
            patience=100, 
            restore_best_weights=True, 
            mode='max', # Added this
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_emotion_class_output_accuracy', 
            factor=0.5, 
            patience=100, 
            min_lr=1e-6, 
            mode='max', # Added this
            verbose=1
        ),
        TensorBoard(log_dir=f'logs/fit/{timestamp}', histogram_freq=1)
    ]

    print("\n--- Starting Model Training ---")
    history = model.fit(
        datasets['train'],
        validation_data=datasets['val'],
        epochs=100,
        callbacks=callbacks,
        verbose=1
    )

    print("\n--- Evaluating Model on Test Set ---")
    if datasets['test']:
        results = model.evaluate(datasets['test'], verbose=1)
        for name, value in zip(model.metrics_names, results):
            print(f"  {name}: {value:.4f}")

    model.save(f'models/final_model_{timestamp}.h5')
    with open(f'models/model_metadata_{timestamp}.json', 'w') as f:
        json.dump(meta, f, indent=4)
    print(f"\nFinal model and metadata saved with timestamp: {timestamp}")

GPU memory growth configured for 1 device(s).


I0000 00:00:1758498892.366783   99856 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5437 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 55,   │          0 │ -                 │
│ (InputLayer)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 128, 55,   │          0 │ input_layer[0][0] │
│                     │ 24, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 128, 55,   │         80 │ lambda[0][0]      │
│ (TimeDistributed)   │ 24, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 128, 55,   │        292 │ time_distributed… │
│ (TimeDistributed)   │ 24, 4)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 128, 5280) │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 128, 32)   │    678,016 │ reshape[0][0]     │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128, 32)   │          0 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 128, 16)   │      2,624 │ dropout[0][0]     │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128, 16)   │          0 │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 16)        │          0 │ dropout_1[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 16)        │        144 │ dropout_1[0][0]   │
│ (Attention)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32)        │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │        528 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16)        │        528 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 16)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 16)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emotion_class_outp… │ (None, 8)         │        136 │ dropout_3[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stressed_not_stres… │ (None, 1)         │         17 │ dropout_2[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 682,365 (2.60 MB)

...


--- Starting Model Training ---
Epoch 1/100


2025-09-22 05:25:03.523101: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91300


42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - emotion_class_output_accuracy: 0.1284 - emotion_class_output_loss: 2.1202 - loss: 2.4068 - stressed_not_stressed_output_accuracy: 0.7112 - stressed_not_stressed_output_loss: 0.5732
Epoch 1: val_emotion_class_output_accuracy improved from None to 0.13889, saving model to models/best_model_20250922-052453.h5


42/42 ━━━━━━━━━━━━━━━━━━━━ 16s 116ms/step - emotion_class_output_accuracy: 0.1084 - emotion_class_output_loss: 2.1193 - loss: 2.4060 - stressed_not_stressed_output_accuracy: 0.7229 - stressed_not_stressed_output_loss: 0.5709 - val_emotion_class_output_accuracy: 0.1389 - val_emotion_class_output_loss: 2.0878 - val_loss: 2.3693 - val_stressed_not_stressed_output_accuracy: 0.7500 - val_stressed_not_stressed_output_loss: 0.5628 - learning_rate: 0.0010
Epoch 2/100
41/42 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - emotion_class_output_accuracy: 0.1196 - emotion_class_output_loss: 2.1177 - loss: 2.4253 - stressed_not_stressed_output_accuracy: 0.7206 - stressed_not_stressed_output_loss: 0.6150
Epoch 2: val_emotion_class_output_accuracy did not improve from 0.13889
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - emotion_class_output_accuracy: 0.1145 - emotion_class_output_loss: 2.1058 - loss: 2.4122 - stressed_not_stressed_output_accuracy: 0.7108 - stressed_not_stressed_output_loss: 0.6063 - val_emotion_clas

  loss: 2.3693
  compile_metrics: 0.5636
  stressed_not_stressed_output_loss: 2.0875
  emotion_class_output_loss: 0.1111

Final model and metadata saved with timestamp: 20250922-052453
